(data-transform)=
# Data Transformation

## Introduction

Visualization is an important tool for generating insight, but it's rare that you get the data in exactly the right form you need to make the graph you want.
Often you'll need to create some new variables or summaries to answer your questions with your data, or maybe you just want to rename the variables or reorder the observations to make the data a little easier to work with.
You'll learn how to do all that (and more!) in this chapter, which will introduce you to data transformation using the **polars** package and a dataset on flights that departed New York City in 2013.

The goal of this chapter is to give you an overview of all the key tools for transforming a data frame.
We'll start with functions that operate on rows and then columns of a data frame, then circle back to talk more about the pipe, an important tool that you use to combine verbs.
We will then introduce the ability to work with groups.
We will end the chapter with a case study that showcases these functions in action and we'll come back to the functions in more detail in later chapters, as we start to dig into specific types of data (e.g., numbers, strings, dates).

### Prerequisites

In this chapter we'll focus on the **polars** package, which is for processing data. It's not actually the most popular package for data analysis in Python: that crown belongs to **pandas**, but polars has a few advantages including its more consistent syntax (which makes it easier to learn!) and the fact that it's a LOT faster in some situations.

You'll need to install **polars** (run `pip install polars` in your terminal if you haven't already).

In [ ]:
import polars as pl

If this command fails, you don't have **polars** installed. Furthermore, if you wish to check which version of **polars** you're using, it's

In [ ]:
pl.__version__

### nycflights13

Let's get some data! Most of the time, data will need to be loaded from a file or the internet. These data are no different, but one of the amazing things about **polars** is how many different types of data it can load, including from files on the internet. The data are 
Let's download the data:

In [ ]:
url = "https://raw.githubusercontent.com/byuidatascience/data4python4ds/master/data-raw/flights/flights.csv"
flights = pl.read_csv(url, null_values="NA")

If the above code worked, then you've downloaded the data in CSV format and put it in a data frame. You may have noticed we used a keyword argument `null_values`. The reason is that this dataset has some fields set to "NA", and we need to tell **polars** to treat these as null (ie missing data) rather than as anything else. Such issues are common with real world datasets!

This dataset contains all flights that departed from New York City in 2013. The data originally come from the US [Bureau of Transportation Statistics](http://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&Link=0).

Let's look at the first few rows using the `.head()` function that works on all **polars** data frames.

In [ ]:
flights.head()

You might have noticed the short abbreviations that appear in the `Dtypes` column. These tell you the type of the values in their respective columns: `int64` is short for integer (eg whole numbers) and `float64` is short for double-precision floating point number (these are real numbers). `object` is a bit of a catch all category for any data type that **pandas** is not really confident about inferring. Although not found here, other data types include `string` for text and `datetime` for combinations of a date and time.

The table below gives some of the most common data types you are likely to encounter.

|  **Name of data type**  |    **Type of data**   |
|:----------:|:-------------:|
|   float64  |  real numbers |
|  category  |   categories  |
| datetime |   date and time  |
|    int64   |    integers   |
|    boolean    | True or False |
|   string   |      text     |
|   Null   |      Invalid or missing value     |

The different column data types are important because the operations you can perform on a column depend so much on its "type"; for example, you can remove all punctuation from strings while you can multiply ints and floats.

We need to do one more thing before our data are ready to use. We would like to work with the `"time_hour"` variable in the form of a datetime; fortunately, **polars** makes it easy to perform that conversion on that specific column:

In [ ]:
flights = flights.with_columns(pl.col("time_hour").str.to_datetime())

### **polars** basics

The two core components of **polars** are *contexts* and *expressions*. We're going to start with contexts. A context, as implied by the name, refers to the context in which an expression needs to be evaluated. There are three main contexts, shown here operating on an arbitrary dataframe `df`.

1. Selection, for example of columns: `df.select([..])`, `df.with_columns([..])`
2. Filtering, for example of rows: `df.filter()`
3. Groupby / Aggregation: `df.groupby(..).agg([..])`

Because each does one thing well, solving complex problems will usually require combining these. This is usually done by *chaining* them together, so you get lines of code like: `df.with_columns([]).filter().agg()`, where the brackets would be filled by the names of columns and variables. The easiest way to think about this is that each time you see a full stop between code, you're saying "then do this". The beauty of this approach is that it makes it easy to read, and to write: even in this very abstract expression you can guess that we're taking a dataframe *then* only using some columns, *then* we're filtering them by a condition, *then* we're aggregating them somehow.

## Filtering

The most important method that operates on rows of a dataset is .`filter()`, which changes which rows are present without changing their order.
This only affects the rows, while the columns are left unchanged. For example, we could find all flights that departed more than 120 minutes (two hours) late.